# nDSM Generator (DSM âˆ’ DGM1 / Bavaria)

This notebook downloads a **DTM (DGM1)** from Bavariaâ€™s INSPIRE **WCS 2.0.1**, clips it to your DSMâ€™s AOI, **aligns** it to the DSM grid, and computes an **nDSM = DSM âˆ’ DTM**.

---

#### What youâ€™ll need
- A **DSM GeoTIFF** (any CRS; will be handled).
- **Credentials** for the Bavarian WCS (LDBV).
- Packages: `requests`, `rasterio`, `shapely`, `numpy`.

> **Data source & license:** DGM1 â€” *Bayerische Vermessungsverwaltung* (LDBV), **CC BY 4.0**. Bitte die Quelle in Ergebnissen angeben.

---

#### Inputs you set in the next cell
- `DSM_PATH` â€“ path to your DSM `.tif`
- `DTM_SAVE_PATH` â€“ where to save the downloaded DTM (raw WCS result; an aligned copy is created automatically)
- `NDSM_SAVE_PATH` â€“ where to save the final nDSM (leave empty to skip)
- `BUFFER_M` â€“ small edge buffer (e.g., 0â€“2 m) to avoid cut-edge artifacts
- `PIXEL_M` â€“ requested output spacing from WCS (DGM1 native is 1 m)

---

#### Outputs produced
- `*_aligned_to_DSM.tif` â€” DTM snapped to **exact DSM grid** (same CRS, transform, size)
- `nDSM_from_DSM_minus_DTM.tif` â€” height of objects above ground (if `NDSM_SAVE_PATH` is set)

---

#### How it works (pipeline)
1. Read DSM extent â†’ transform to **EPSG:25832** â†’ apply small buffer.
2. WCS 2.0.1 **GetCoverage** for `EL.ElevationGridCoverage` with that AOI.
3. **Align** DTM to the DSMâ€™s pixel grid (CRS, transform, dimensions).
4. Compute **nDSM = DSM âˆ’ aligned DTM**.

---

#### Credentials (recommended: env vars)
Set once in your conda env so you donâ€™t type them each time.

**Anaconda:**
```bash
conda activate nDSM_generator
conda env config vars set LDBV_USER="your_user" LDBV_PASS="your_password"
conda deactivate && conda activate nDSM_generator


In [ ]:
from modules.ndsm_tools import *
import os, getpass

In [ ]:
# === USER INPUTS ===
DSM_PATH = r"D:\Data\example_DSM.tif"
DTM_SAVE_PATH = r"D:\Data\example_DTM.tif" #if DTM is required
NDSM_SAVE_PATH = r"D:\Data\example_nDSM.tif"  # "" to skip nDSM

BUFFER_M = 2.0   # 0..2 m is fine for most use cases
PIXEL_M  = 1.0   # DGM1 native spacing (m)

# === Credentials ===
USER = os.getenv("LDBV_USER") or input("LDBV username: ")
PWD  = os.getenv("LDBV_PASS") or getpass.getpass("LDBV password: ")


In [ ]:
ndsm_arg = NDSM_SAVE_PATH if NDSM_SAVE_PATH else None
aligned_dtm, ndsm = run_pipeline(
    dsm_path=DSM_PATH,
    dtm_save_path=DTM_SAVE_PATH,
    ndsm_save_path=ndsm_arg,
    buffer_m=BUFFER_M,
    pixel_m=PIXEL_M,
    user=USER,
    password=PWD,
    allow_prompt=False  # we already supplied creds above
)
print("Aligned DTM:", aligned_dtm)
print("nDSM:", ndsm)
